<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_rec_sys/blob/main/coursework/cw_rec_sys_glvv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Курсовик по предмету "Рекомендательные системы" Гладышев В.В.

### EDA, фильтрация, feature engineering:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/coursework/cw_eda_glvv.ipynb

### библиотека с функциями метрик:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/metrics.py

### подготовка данных: 

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/utils.py

### класс с моделями:

https://github.com/VitalyGladyshev/gb_rec_sys/blob/main/src/recommenders.py

In [2]:
pip install implicit

     |████████████████████████████████| 1.1MB 15.3MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419393 sha256=3aba024f18a0458f7d088b20accf3a0035f333e5006e3700741898650bab5817
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


In [3]:
pip install lightfm

     |████████████████████████████████| 317kB 16.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp36-cp36m-linux_x86_64.whl size=704269 sha256=21830e58157f9a357165633e73ab410200057d070786318774107eee573eece6
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
path_to_src = '/content/gdrive/My Drive/Colab Notebooks/GB Rec Sys/src/'

In [7]:
if path_to_src not in sys.path:
    sys.path.append(path_to_src)

# Написанные функции
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [8]:
path_to_files = '/content/gdrive/My Drive/Colab Notebooks/GB Rec Sys/data'

In [9]:
data          = pd.read_csv(os.path.join(path_to_files, "data.csv"))
user_features = pd.read_csv(os.path.join(path_to_files, "user_features.csv"))
item_features = pd.read_csv(os.path.join(path_to_files, "item_features.csv"))

In [10]:
# схема обучения и валидации
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,999999,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,999999,1,0.82,364,0.0,1631,1,0.0,0.0


In [11]:
recommender = MainRecommender(data_train_lvl_1, item_features, user_features)

user_item_matrix
user_feat
item_feat


GPU training requires factor size to be a multiple of 32. Increasing factors from 50 to 64.


ALS model


ALS TF-IDF model


ALS BM25 model
LightFM model
Own


In [12]:
users_lvl_2_pred_als = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_lvl_2_pred_als.columns = ['user_id']

In [220]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

In [221]:
k_n = 5
num = 200

In [15]:
popularity = data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
item_list = popularity.sort_values('n_sold', ascending=False).head(num+1).item_id.tolist()
item_list = item_list[1:num+1]
len(item_list)

200

In [16]:
user_train_list = data_train_lvl_1["user_id"].unique().tolist()

In [17]:
def get_pred_als(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [18]:
def get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_tfidf_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [19]:
def get_pred_als_bm25(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_als_bm25_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [20]:
def get_pred_own(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_own_recommendations(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [21]:
def get_pred_similar_items(recommender, user_train_list, item_list, x, num=5):
    res = []
    if x in user_train_list:
        res.extend(recommender.get_similar_items_recommendation(x, rec_num=num))
    else:
        res.extend(item_list)    # res.extend(random.choices(item_list, k=num))
    return res

In [22]:
users_lvl_2_pred_als['predictions'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec"].mean()

0.2190009794319263

In [23]:
users_lvl_2_pred_als["rec"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec"].mean()

0.039716490150321734

In [23]:
users_lvl_2_pred_als['predictions_tfidf'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_tfidf(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_tfidf"].mean()

0.15945151811948832

In [24]:
users_lvl_2_pred_als["rec_tfidf"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_tfidf"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_tfidf"].mean()

0.029782777371396233

In [25]:
users_lvl_2_pred_als['predictions_bm25'] = users_lvl_2_pred_als['user_id'].apply(\
                            lambda x: get_pred_als_bm25(recommender, user_train_list, item_list, x, num=num))
    
users_lvl_2_pred_als["prec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: precision_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=k_n))

users_lvl_2_pred_als["prec_bm25"].mean()

0.21057786483839058

In [26]:
users_lvl_2_pred_als["rec_bm25"] = users_lvl_2_pred_als['user_id'].apply(\
        lambda x: recall_at_k(users_lvl_2_pred_als.loc[users_lvl_2_pred_als["user_id"] == x, "predictions_bm25"].values[0], 
                              list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                              k=k_n))

users_lvl_2_pred_als["rec_bm25"].mean()

0.03214532586471245

In [27]:
users_lvl_2_pred_als.head()

,user_id,predictions,prec,rec,predictions_tfidf,prec_tfidf,rec_tfidf,predictions_bm25,prec_bm25,rec_bm25
0,338,"[845208, 1037863, 904360, 981760, 1026118, 108...",0.4,0.026667,"[845208, 923746, 1037863, 904360, 962568, 1026...",0.2,0.013333,"[1082185, 981760, 845208, 1026118, 895166, 871...",0.4,0.026667
1,2120,"[1082185, 1106523, 981760, 1029743, 995242, 11...",0.0,0.000000,"[1082185, 1029743, 995242, 981760, 1127831, 11...",0.0,0.000000,"[1082185, 981760, 989824, 995055, 1041476, 109...",0.0,0.000000
2,2324,"[995242, 1082185, 1029743, 5978656, 981760, 91...",0.6,0.071429,"[1005186, 1136257, 1039156, 5978656, 1082185, ...",0.4,0.047619,"[1082185, 981760, 5978656, 995242, 1127831, 84...",0.4,0.047619
3,514,"[878996, 1082185, 1024306, 866211, 1127831, 10...",0.8,0.125000,"[1062002, 1127179, 878996, 1082185, 1110843, 9...",0.8,0.125000,"[1082185, 878996, 1062002, 1127831, 1023720, 8...",0.6,0.093750
4,1762,"[1110572, 839419, 1068719, 971949, 986912, 826...",0.0,0.000000,"[1110572, 845705, 1084331, 1053690, 839419, 10...",0.0,0.000000,"[1029743, 826249, 1098066, 1067606, 995242, 10...",0.2,0.011765


### Строим классификатор

In [140]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: get_pred_als(recommender, user_train_list, item_list, x, num=num))

In [141]:
users_lvl_2.head()

,user_id,candidates
0,2070,"[1107553, 5569471, 833025, 1096036, 1029743, 9..."
1,2021,"[1082185, 951590, 981760, 1044078, 844179, 826..."
2,1753,"[1085604, 1005186, 879755, 1106523, 986912, 11..."
3,2120,"[1082185, 1106523, 1029743, 981760, 995242, 10..."
4,1346,"[1082185, 1127831, 866211, 1058997, 878996, 11..."


In [142]:
users_lvl_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     2154 non-null   int64 
 1   candidates  2154 non-null   object
dtypes: int64(1), object(1)
memory usage: 33.8+ KB


In [222]:
df=pd.DataFrame({'user_id':users_lvl_2.user_id.values.repeat(len(users_lvl_2.candidates[0])),
                 'item_id':np.concatenate(users_lvl_2.candidates.values)})

In [223]:
targets_lvl_2 = data_train_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = df.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)

In [224]:
targets_lvl_2.shape

(444800, 3)

In [225]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left') #
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left') #

targets_lvl_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,bill_avg,week_last_bill,items_per_week
0,2070,1107553,0.0,103,GROCERY,National,SOFT DRINKS,SFT DRNK SNGL SRV BTL CARB (EX,20 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815
1,2070,5569471,0.0,1208,GROCERY,National,SOFT DRINKS,SOFT DRINKS 12/18&15PK CAN CAR,12 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,2.891039,95.0,29.814815


In [226]:
targets_lvl_2.shape

(444800, 19)

In [227]:
targets_lvl_2.fillna(0, inplace=True)

In [228]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [229]:
cat_feats = X_train.columns[2:].tolist()
X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'bill_avg',
 'week_last_bill',
 'items_per_week']

In [230]:
lgb = LGBMClassifier(objective='binary', 
                     n_estimators=1500,
                     learning_rate=0.008,
                     max_depth=7, 
                     verbose=1,
                     categorical_column=cat_feats)

hist = lgb.fit(X_train, y_train, 
               eval_set=[(X_train, y_train)],
               eval_metric=['logloss'],
               early_stopping_rounds=50,
               verbose=0)

train_preds = lgb.predict_proba(X_train)

In [231]:
hist.evals_result_['training']['binary_logloss'][-10:]

[0.2396841878127636,
 0.23967181350538252,
 0.23965332173955894,
 0.23963101027024172,
 0.239619345669979,
 0.23960667120520351,
 0.23959363405947803,
 0.23958299230636929,
 0.23957112226368338,
 0.2395491156431475]

In [232]:
classifer_prediction = X_train[['user_id', 'item_id']]
classifer_prediction["pred"] = train_preds[:, 1]
classifer_prediction.tail()

,user_id,item_id,pred
444795,1745,907631,0.012037
444796,1745,858302,0.010879
444797,1745,965267,0.001061
444798,1745,1100972,0.013453
444799,1745,10204822,0.000847


In [233]:
res_val = classifer_prediction.loc[classifer_prediction["pred"]>0.5, ["user_id", "item_id"]]
res_val.shape

(9976, 2)

In [234]:
pred_lvl_2 = res_val.groupby('user_id')['item_id'].unique().reset_index()
pred_lvl_2.columns=['user_id', 'pred']
pred_lvl_2.head()

,user_id,pred
0,1,"[995242, 1029743, 1082185, 9655212, 1049998, 1..."
1,7,[1082185]
2,8,[1082185]
3,13,"[995242, 5978656, 1029743, 5978648, 1082185, 1..."
4,16,"[1082185, 1313189]"


In [235]:
result_lvl_2 = result_lvl_2.merge(pred_lvl_2, on='user_id', how='left')
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[995242, 1029743, 1082185, 9655212, 1049998, 1..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...",NaN
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...",NaN
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...",[1082185]
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...",[1082185]


In [236]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["pred"] != result_lvl_2["pred"], 'user_id'].values
len(user_pred_nan)

1384

In [237]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2498

In [238]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

1383

In [239]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "pred"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

,user_id,actual,pred
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[995242, 1029743, 1082185, 9655212, 1049998, 1..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 951590, 1053690, 1106523, 5568378]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[878996, 866211, 1127831, 923746, 1023720]"
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...",[1082185]
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...",[1082185]


In [240]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...",NaN


In [241]:
popularity = recommender.prep_data.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top_5 = popularity.sort_values('n_sold', ascending=False).head(6).item_id.tolist()[1:6]
top_5

[1082185, 995242, 1029743, 1133018, 981760]

In [242]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"] =\
        result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "user_id"].apply(lambda x: top_5)

In [243]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,pred
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...","[1082185, 995242, 1029743, 1133018, 981760]"


In [244]:
len(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])

5

In [245]:
res = []
res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == 1984, "pred"].values[0])
res

[1082185, 995242, 1029743, 1133018, 981760]

In [246]:
def fill_if_not_5(recommender, x):
    res = []
    res.extend(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    pred_num = len(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0])
    if pred_num < 5:
        res.extend(recommender.get_als_recommendations(x, rec_num=5-pred_num))
    return res

In [249]:
# result_lvl_2["pred"] = result_lvl_2["user_id"].apply(lambda x: fill_if_not_5(recommender, x))

# result_lvl_2.head()

In [248]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "pred"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

0.2761426705843928

### LightGBM Ranker

In [118]:
X_train.shape

(444800, 18)

In [119]:
X_train['num'] = np.arange(len(X_train))

In [120]:
gr = X_train[['user_id', 'num']].groupby('user_id', sort=False).max()['num'].values

In [121]:
grs = np.concatenate((np.array([gr[0]+1]), gr[1:] - gr[:-1]))

In [122]:
X_train.shape, y_train.shape, np.sum(grs)

((444800, 19), (444800, 1), 444800)

In [123]:
from lightgbm import LGBMRanker

lgb_params = { 
    'objective':'lambdarank',
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.08,
    'max_depth': 5,
    'categorical_column': cat_feats,
    'random_state': 27,
    'verbose': 0,
    'is_unbalance': True
}

In [124]:
lgb_rn = LGBMRanker(**lgb_params, silent=False)

eval_h = lgb_rn.fit(X_train, y_train, group=grs,
                    eval_set=[(X_train, y_train)], eval_group=[grs], 
                    eval_metric=['ndcg'],
                    eval_at=[5, 10 ], 
                    early_stopping_rounds=50, 
                    verbose=0)

In [125]:
eval_h.evals_result_['training']['ndcg@5'][:3], eval_h.evals_result_['training']['ndcg@5'][-3:]

([0.4831570404401664, 0.4831570404401664, 0.47107170366830964],
 [0.4840221018323034, 0.48397635794464283, 0.48397635794464283])

In [126]:
rank_preds = lgb_rn.predict(X_train)

In [127]:
ranker_prediction = X_train[['user_id', 'item_id']]
ranker_prediction["pred"] = rank_preds
ranker_prediction.head()

,user_id,item_id,pred
0,2070,1107553,0.000060
1,2070,5569471,-0.004316
2,2070,833025,-0.019442
3,2070,1096036,-0.004316
4,2070,1029743,0.034782


In [128]:
ranker_prediction = ranker_prediction.drop_duplicates()

In [129]:
ranker_prediction.sort_values(by="pred", inplace=True, ascending=False)
ranker_prediction.head()

,user_id,item_id,pred
226051,1225,1082185,0.034782
52637,628,1126899,0.034782
52618,628,1070820,0.034782
258549,1057,1082185,0.034782
412162,1772,1029743,0.034782


In [130]:
ranked_res = ranker_prediction.groupby('user_id')['item_id'].unique().reset_index()   
ranked_res

,user_id,item_id
0,1,"[1029743, 1082185, 1070820, 995242, 981760, 86..."
1,2,"[1106523, 1082185, 1029743, 1133018, 995242, 9..."
2,4,"[1082185, 1029743, 1106523, 1126899, 1058997, ..."
3,6,"[1070820, 1106523, 1029743, 1082185, 995242, 1..."
4,7,"[1126899, 1082185, 1106523, 1029743, 1058997, ..."
...,...,...
2149,2496,"[1106523, 1082185, 1133018, 981760, 5978648, 8..."
2150,2497,"[1070820, 1082185, 1029743, 995242, 981760, 98..."
2151,2498,"[1082185, 1029743, 1126899, 1070820, 995242, 1..."
2152,2499,"[1106523, 1082185, 1070820, 1133018, 981760, 8..."


In [133]:
result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(ranked_res, on='user_id', how='left')
result_lvl_2

,user_id,actual,item_id
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[1029743, 1082185, 1070820, 995242, 981760, 86..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...",NaN
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[1070820, 1106523, 1029743, 1082185, 995242, 1..."
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1082185, 1106523, 1029743, 1058997, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1029743, 1082185, 1070820, 1106523, 995242, 1..."
...,...,...,...
2037,2496,[999999],"[1106523, 1082185, 1133018, 981760, 5978648, 8..."
2038,2497,"[1016709, 9835695, 999999, 845294, 871756, 873...","[1070820, 1082185, 1029743, 995242, 981760, 98..."
2039,2498,"[999999, 901776, 914190, 958382, 972437, 10398...","[1082185, 1029743, 1126899, 1070820, 995242, 1..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[1106523, 1082185, 1070820, 1133018, 981760, 8..."


In [134]:
user_pred_nan = result_lvl_2.loc[result_lvl_2["item_id"] != result_lvl_2["item_id"], 'user_id'].values
len(user_pred_nan)

126

In [135]:
set_tr_1 = set(data_train_lvl_1.user_id.unique())
len(set_tr_1)

2498

In [136]:
user_list = set(user_pred_nan).intersection(set_tr_1)
len(user_list)

126

In [137]:
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "item_id"] =\
result_lvl_2.loc[result_lvl_2["user_id"].isin(user_list), "user_id"].apply(lambda x: recommender.get_als_recommendations(x, rec_num=k_n))
result_lvl_2.head()

,user_id,actual,item_id
0,1,"[999999, 856942, 865456, 889248, 907957, 91419...","[1029743, 1082185, 1070820, 995242, 981760, 86..."
1,3,"[835476, 999999, 872021, 909638, 920626, 95815...","[1092026, 951590, 1053690, 1106523, 5568378]"
2,6,"[999999, 946489, 1006718, 1104227, 1108624, 11...","[1070820, 1106523, 1029743, 1082185, 995242, 1..."
3,7,"[840386, 999999, 898068, 909714, 993838, 10031...","[1126899, 1082185, 1106523, 1029743, 1058997, ..."
4,8,"[835098, 872137, 910439, 924610, 992977, 99999...","[1029743, 1082185, 1070820, 1106523, 995242, 1..."


In [138]:
result_lvl_2.loc[result_lvl_2["user_id"] == 1984]

,user_id,actual,item_id
1622,1984,"[999999, 847344, 859237, 1064380, 1074516, 122...","[1070820, 1126899, 1106523, 1029743, 1082185, ..."


In [139]:
result_lvl_2["rec"] = result_lvl_2['user_id'].apply(\
        lambda x: precision_at_k(result_lvl_2.loc[result_lvl_2["user_id"] == x, "item_id"].values[0], 
                                 list(result_lvl_2.loc[result_lvl_2["user_id"] == x, "actual"].values[0]), 
                                 k=5))

result_lvl_2['rec'].mean()

0.20979431929480583